# Example API Usage

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pathlib
import tempfile

import numpy as np

from ska_pydada import AsciiHeader, DadaFile

## Create a DADA file

The cells below show how to create a DADA file with some random data.

The steps to create a DADA file are:
* create a header
* set the data
* serialise/dump to an output file

### Create ASCII Header

A header for the DADA file format is a simple key value structure that has at least 4096 bytes. The size
of the ASCII header is defined by the `HDR_SIZE` key and in the PyDADA library one can override this value
at construction time or via setting the property later.

```python
header = AsciiHeader(header_size=16384)
```

or

```python
header = AsciiHeader()
header.header_size = 16384
```

in the either example above the serialised header will have exactly 16384 bytes.

A header can be created from a string or a byte array (this is what is used when the `DadaFile` loads a DADA file).  The following
will use the `header_txt` variable to load the header.

In [3]:
header_txt = """HDR_SIZE            16384
HDR_VERSION         1.0
NCHAN               432
NBIT                32
NDIM                2
NPOL                2
RESOLUTION          1327104
UTC_START           2017-08-01-15:53:29
"""

In [4]:
header = AsciiHeader.from_str(header_txt)

In [5]:
assert header.header_size == 16384
assert int(header["NCHAN"]) == 432
assert int(header["NBIT"]) == 32
assert int(header["NDIM"]) == 2
assert int(header["NPOL"]) == 2
assert header.resolution == 1327104
assert header["UTC_START"] == "2017-08-01-15:53:29"

There are also utility methods on the `AsciiHeader` to get a header record as an `int` or `float`.

In [6]:
nbit = header.get_int("NBIT")
assert nbit == 32

Values can be added to the header either using a Python `dict` item insert or using the `set_value`

In [7]:
header["SOURCE"] = "J1644-4559_R"
assert header.get_value("SOURCE") == "J1644-4559_R"

# or

header.set_value("DESCRIPTION", "Some fancy description")
assert header["DESCRIPTION"] == "Some fancy description"

### Generate some data

For this notebook, the data will be random complex data with 768 time bins, 432 frequency channels, and 2 polarisations.

In [8]:
data = np.random.rand(768, 432, 2 * 2).astype(np.float32).view(np.complex64)
data.shape

(768, 432, 2)

In [9]:
data

array([[[0.571237  +0.15780433j, 0.5507741 +0.66138965j],
        [0.00513218+0.6092555j , 0.41201785+0.33653668j],
        [0.12717463+0.95208985j, 0.04352839+0.3474904j ],
        ...,
        [0.68198854+0.23360793j, 0.60173047+0.89112836j],
        [0.24145234+0.0081375j , 0.35247105+0.12637012j],
        [0.61966825+0.6126681j , 0.10123057+0.8272245j ]],

       [[0.90665036+0.22999409j, 0.6311375 +0.39842412j],
        [0.2927027 +0.7986666j , 0.7159421 +0.35195136j],
        [0.1886648 +0.6309893j , 0.36530688+0.5790974j ],
        ...,
        [0.85565263+0.13946877j, 0.18159713+0.8473413j ],
        [0.674901  +0.6955593j , 0.31912088+0.64478064j],
        [0.16821766+0.9779457j , 0.9796789 +0.32850793j]],

       [[0.5748353 +0.13305499j, 0.95649505+0.5545998j ],
        [0.03715614+0.05125557j, 0.04500831+0.02207299j],
        [0.91846144+0.5616338j , 0.7528865 +0.43210357j],
        ...,
        [0.6154221 +0.4247898j , 0.721579  +0.771155j  ],
        [0.45945323+0.4146547

An instance of a `DadaFile` can be created using the constructor that takes an optional `AsciiHeader` and an optional by array of data.

However, the following show how to create a `DadaFile` and then set the data afterwards.

In [10]:
dada_file = DadaFile(header=header)
dada_file.set_data(data)

Once an instance of a `DadaFile` has been created, it can be saved as a file. This can be then be read back later

In [11]:
assert dada_file.data_size == len(data.tobytes())
dada_file.data_size

5308416

In [12]:
tmpdir = tempfile.gettempdir()
outfile = pathlib.Path(tmpdir) / "example_dada_file.dada"

dada_file.dump(outfile)

In [13]:
%ls -lh $outfile

-rw-rw-r-- 1 wgauvin wgauvin 5.1M May 29 10:27 /tmp/example_dada_file.dada


In [14]:
!head -c4096 $outfile

HDR_SIZE            16384
HDR_VERSION         1.0
NCHAN               432
NBIT                32
NDIM                2
NPOL                2
RESOLUTION          1327104
UTC_START           2017-08-01-15:53:29
SOURCE              J1644-4559_R
DESCRIPTION         Some fancy description
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

## Loading and reading DADA files

While the above shows how to create DADA files that is normally used for testing and is not the main focus of the PyDADA library or `DadaFile` itself. The power of the `DadaFile` is that it can read DADA files that conform to the DADA spec of having a header that has `HDR_SIZE` value of at least 4096 bytes and that the binary data is afterwards. As it is a flexible file format the data may be packed in different ways and this API provides general data access methods to get the data.

The following cell will read the file generated above and print out the header.

In [15]:
dada_file2 = DadaFile.load_from_file(outfile)
print(dada_file2.header)

HDR_SIZE            16384
HDR_VERSION         1.0
NCHAN               432
NBIT                32
NDIM                2
NPOL                2
RESOLUTION          1327104
UTC_START           2017-08-01-15:53:29
SOURCE              J1644-4559_R
DESCRIPTION         Some



To get the data in time, frequency and polarisation structure, once can use the `as_time_freq_pol` help method. There is no need to know how the data is layed out provided the header records are correct.

In [16]:
tfp_data = dada_file2.as_time_freq_pol()
tfp_data

array([[[0.571237  +0.15780433j, 0.5507741 +0.66138965j],
        [0.00513218+0.6092555j , 0.41201785+0.33653668j],
        [0.12717463+0.95208985j, 0.04352839+0.3474904j ],
        ...,
        [0.68198854+0.23360793j, 0.60173047+0.89112836j],
        [0.24145234+0.0081375j , 0.35247105+0.12637012j],
        [0.61966825+0.6126681j , 0.10123057+0.8272245j ]],

       [[0.90665036+0.22999409j, 0.6311375 +0.39842412j],
        [0.2927027 +0.7986666j , 0.7159421 +0.35195136j],
        [0.1886648 +0.6309893j , 0.36530688+0.5790974j ],
        ...,
        [0.85565263+0.13946877j, 0.18159713+0.8473413j ],
        [0.674901  +0.6955593j , 0.31912088+0.64478064j],
        [0.16821766+0.9779457j , 0.9796789 +0.32850793j]],

       [[0.5748353 +0.13305499j, 0.95649505+0.5545998j ],
        [0.03715614+0.05125557j, 0.04500831+0.02207299j],
        [0.91846144+0.5616338j , 0.7528865 +0.43210357j],
        ...,
        [0.6154221 +0.4247898j , 0.721579  +0.771155j  ],
        [0.45945323+0.4146547

In [17]:
np.testing.assert_allclose(tfp_data, data)

The TFP data can also be retrieved by using the `data_c64` method one should include the shape.

In [18]:
tfp_data2 = dada_file2.data_c64(shape=(-1, 432, 2))
tfp_data2.shape

(768, 432, 2)

In [19]:
np.testing.assert_allclose(tfp_data2, data)

The raw data can be retrieved from the file by using:

In [20]:
raw_data = dada_file2.raw_data
len(raw_data), raw_data[:20]

(5308416, b'\x97<\x12?u\x97!>\x88\xff\x0c?\xd5P)?\xdf+\xa8;')

### Large files

This notebook as uses small files but data recorded during a scan can result in large files.  Loading the whole file into memory is not efficient
and the `DadaFile` defaults to only loading data of around 4MB at a time, the amount of data loaded will be equal to a multiple of `RESOLUTION` value defined in the header (this does default to 1 byte if not set).

The below shows using the `load_next` method to get the next lot of data.

In [21]:
data = np.random.rand(768 * 2, 432, 2 * 2).astype(np.float32).view(np.complex64)
data.shape

dada_file.set_data(data)
dada_file.dump(outfile)

/tmp/example_dada_file.dada already exists, overwriting it


In [22]:
%ls -lh $outfile

-rw-rw-r-- 1 wgauvin wgauvin 11M May 29 10:27 /tmp/example_dada_file.dada


In [23]:
dada_file3 = DadaFile.load_from_file(outfile)
len(dada_file3.raw_data)

5308416

In [24]:
raw_data1 = dada_file3.raw_data
bytes_read = dada_file3.load_next()
assert bytes_read == 5308416
raw_data2 = dada_file3.raw_data

np.testing.assert_raises(AssertionError, np.testing.assert_array_equal, raw_data1, raw_data2)

bytes_read = dada_file3.load_next()
assert bytes_read == 0
raw_data3 = dada_file3.raw_data
np.testing.assert_array_equal(raw_data2, raw_data3)

In [25]:
raw_data1[:10], raw_data2[:10], raw_data3[:10]

(b'\x90\x1e/?\xbeU0?\xbfH',
 b'{\x94\x9d=\xbdd\xd4>5\x8e',
 b'{\x94\x9d=\xbdd\xd4>5\x8e')

From the above raw data we see that first call to `load_next` returns new data but the next call doesn't.

Methods like the `as_time_freq_pol` can still be used after a read but it is on the latest chuck of data.

In [26]:
tfp = dada_file3.as_time_freq_pol()
tfp

array([[[0.07694336+0.41483107j, 0.75607616+0.45915896j],
        [0.5186104 +0.32094872j, 0.2738332 +0.41057816j],
        [0.23330808+0.9626105j , 0.33791602+0.11652099j],
        ...,
        [0.43164793+0.12489458j, 0.49463692+0.18952423j],
        [0.29684052+0.09080667j, 0.0617302 +0.909739j  ],
        [0.7257428 +0.9792754j , 0.83314866+0.80731326j]],

       [[0.8015701 +0.59756005j, 0.04048614+0.42602417j],
        [0.935496  +0.06028024j, 0.38036686+0.33810487j],
        [0.3278966 +0.65708905j, 0.78201854+0.39740697j],
        ...,
        [0.04617917+0.6917416j , 0.46211886+0.19552189j],
        [0.853596  +0.22222199j, 0.63520795+0.31013742j],
        [0.28625035+0.8315675j , 0.8378862 +0.27095002j]],

       [[0.2864877 +0.5696607j , 0.5336498 +0.31873134j],
        [0.69987416+0.8328891j , 0.53243315+0.1968386j ],
        [0.65483636+0.9235317j , 0.45626995+0.9593825j ],
        ...,
        [0.39700472+0.53900254j, 0.74282795+0.18763994j],
        [0.87814873+0.8659404

Note that the length of the raw dada is just over 5MB of data (it is `4 * RESOLUTION`). However, the file is around 11MB in size. More data can be loaded and processed by using the `load_next`

### Loading all the data

If you really need to load all the data in one go, you can pass a chunk size of `< 0` (i.e. -1) when creating the file.

In [27]:
dada_file4 = DadaFile.load_from_file(outfile, chunk_size=-1)

dada_file4.data_size

10616832

In [28]:
data = dada_file4.as_time_freq_pol()
data.shape

(1536, 432, 2)

## Clean up

In [29]:
if outfile.exists():
    outfile.unlink()